In [1]:
import segyio
import torch
import numpy as np
import matplotlib.pyplot as plt
import csv
import os
from sgykit import Sgykit
from config import Config
from generatadata import save_dataT, save_data
from dataset import SingleSample
from checkdata import CheckData
from resunet3d import Res18Unet3D
from unet3d import Unet3D
from evaluate import Evaluate

In [4]:
cfg = Config()
checker = CheckData()
evlter = Evaluate()
unet3d = Unet3D(cfg).to(cfg.device)
unet3d.load_state_dict(torch.load('checkpoint/unet/0best.mdl'))
# unet3d = Res18Unet3D(cfg).to(cfg.device)
# unet3d.load_state_dict(torch.load('checkpoint/resunet/best.mdl'))
fileName = 'D:\Work\Project\Geomodeling\CB\CB_depth\sample.segy'
# fileName = 'D:\Work\Project\Geomodeling\CB\CB_depth\Chengbei_2020_3D_psdm_final_depth.segy'
# fileName = 'D:\Work\Project\Geomodeling\CB\CB_depth\Chengbei_C3.sgy'
sgykit = Sgykit(fileName,
                sliceSize={'iline': 128, 'xline': 128, 'time': 128},)

sgy shape is : (210, 381, 1001)
Inline can be sliced 1 section. There are 82 remainder.
Crossline can be sliced 2 section. There are 125 remainder.
Time can be sliced 7 section. There are 105 remainder.
Blocks is 14


In [5]:
sgykit.sgy_detail(fileName)

(80010, 1001)
(x,y) locations: 80010 80010
troce numbers: (80010,)
sample_interval: 4.0 ms
C1  BEGIN EBCDIC LINE HEADER                                                    
C2                                                                              
C3  Data generated by: AASPI, The University of Oklahoma, Norman, OK, USA       
C4  File generated on 04/13/2016 at time 11:12                                  
C5                                                                              
C6  value of 1st samp in s    samp incr in 1.E-6*s            no. of samples    
C7                     0.000                    4000                    1001    
C8  binary input AASPI format file name =                                       
C9  /ouhomes2/qi1400/image_processing/d_mig_westcam.H                           
C10     first line no.     last line no.   line index incr  line incr in m      
C11                181               390                 1            25.011    
C12      first cdp

In [4]:
data = sgykit.read3D(fileName)
print(type(data), data.dtype, data.shape)

<class 'numpy.ndarray'> float32 (210, 381, 1001)


In [5]:
path = 'data/sample/seis/'
gendb = sgykit.split_sgy(data, path,
                        #  False,
                         True,
                         size={'i': 128, 'x': 128, 't': 128},
                         bias={'i': 40, 'x': 0, 't': 0})
save_data(gendb, 'data/sample/seis/', 'gendb.datt')
print(gendb.shape)

(128, 256, 896)


In [7]:
redb = sgykit.reconstruct_sgy('data/sample/seis/')
checker.check_slice_data(gendb, redb)

True

In [6]:
sgset = SingleSample('seis', 'data/sample/seis/', (128, 128, 128), 1, '.dat')

initialization successful! This folder hive 14.dat


In [9]:
unet3d.eval()
with torch.no_grad():
    for i in range(sgset.fileNums):
        sgdata = sgset.getitem(i).to(cfg.device)
        pred = unet3d(sgdata)
        # print(type(pred), pred.shape)
        prednp = evlter._flatten2np(pred)
        save_data(prednp, 'data/sample/pred/resunet/', f'{i}.dat')
        save_data(prednp, 'data/sample/pred/resunet/', f'{i}.dat')
        # break

In [10]:
redb = sgykit.reconstruct_sgy('data/sample/pred/resunet/')

In [11]:
save_data(redb, 'data/sample/pred/resunet/', 'predb.datt')

  - 去边缘效应

In [55]:
import numpy as np
from scipy.ndimage import gaussian_filter

def get_gaussian(s=(128, 128, 128), sigma=1.0/8) -> np.ndarray:
	temp = np.zeros(s)
	# print('temp:', temp)
	# print('sigma:', sigma)
	coords = [i // 2 for i in s]
	# print('coords:', coords)
	sigmas = [i * sigma for i in s]
	# print('sigmas:', sigmas)
	temp[tuple(coords)] = 1
	# print('tuple(coords):', tuple(coords))
	# print('temp[tuple(coords)]:', temp[(1, 1)])
	gaussian_map = gaussian_filter(temp, sigmas, 0, mode='constant', cval=0)
	gaussian_map /= np.max(gaussian_map)
	return gaussian_map

# get_gaussian()

In [60]:
def smooth_sgy(
        data:np.ndarray,
        # fileName,
        # dataSize,
        patchSize=(128, 128, 128)):
    
    patchSize = np.array(patchSize)
    patchStride = patchSize // 2
    # data = np.fromfile(fileName, dtype=np.single)
    # data = data.reshape(dataSize)

    result = np.zeros(data.shape)
    normalization = np.zeros(data.shape)
    gaussian_map = get_gaussian(patchSize)
    for i in range(0, data.shape[0]-patchSize[0]+1, patchStride[0]):
        for j in range(0, data.shape[1]-patchSize[1]+1, patchStride[1]):
            for k in range(0, data.shape[2]-patchSize[2]+1, patchStride[2]):
                patch = data[i:i+patchSize[0],
                             j:j+patchSize[1],
                             k:k+patchSize[2]].astype(np.float32)
                patch *= gaussian_map
                normalization[i:i+patchSize[0],
                              j:j+patchSize[1],
                              k:k+patchSize[2]] += gaussian_map
                result[i:i+patchSize[0],j:j+patchSize[1],k:k+patchSize[2]] += patch
    result /= normalization
    return result.astype(np.single)
# plt.imshow(result,cmap=plt.get_cmap('gray'))
# plt.xticks([])
# plt.yticks([])
# plt.show()

In [61]:
path = 'data/sample/pred/predb.datt'
data = np.fromfile(path, dtype=np.single)
print(data.shape)
data = data.reshape(128, 256, 896)
print(data.shape)
print(data.max())
data1 = smooth_sgy(data)
print(data1.shape)
print(data1.max())

(29360128,)
(128, 256, 896)
0.99972767
(128, 256, 896)
0.99972767


In [62]:
data1.tofile('data/sample/pred/predb_s.datt')
data2 = np.fromfile('data/sample/pred/predb_s.datt', dtype=np.single)
print(data2.shape)

(29360128,)


In [15]:
import torch

path = 'data/sample/pred/resunet/predb.datt'
data = np.fromfile(path, dtype=np.single)
data = torch.tensor(data)
data1 = torch.where(data>0.7, data, 0)
data1 = data1.numpy()
data1.tofile('data/sample/pred/resunet/predb70.datt')

In [69]:
path = 'data/sample/pred/predb80.datt'
data = np.fromfile(path, dtype=np.single)
data1 = data.reshape(128, 256, 896)
print(data2.shape)
data2 = smooth_sgy(data1)
data2.tofile('data/sample/pred/predb80_s.datt')

(29360128,)


- 取单个样本

In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import os, hashlib, csv, glob
from evaluate import Evaluate
from torch.utils.data import Dataset, DataLoader
from torch import optim
from config import Config
from unet3d import Unet3D, BalancedCrossEntropyLoss, BatchBalancedCrossEntropyLoss
from resunet3d import Res18Unet3D
from dataset import FaultData, SingleSample
from checkdata import CheckData
evlat = Evaluate()
cfg = Config()
checker = CheckData()

In [2]:
# unet3d = Unet3D(cfg).to(cfg.device)
unet3d = Res18Unet3D(cfg).to(cfg.device)
unet3d.load_state_dict(torch.load('checkpoint/resunet/best.mdl'))
optimizer = optim.Adam(unet3d.parameters(), lr=cfg.lr)
# criteon = nn.BCELoss()
# criteon = nn.CrossEntropyLoss().to(cfg.device)
# criteon = BalancedCrossEntropyLoss().to(cfg.device)
criteon = BatchBalancedCrossEntropyLoss().to(cfg.device)

In [6]:
path = './data/train/0seis/'
id = 100
ss = SingleSample('seis', path).getitem(id).to(cfg.device)
print(ss.shape, ss.type())

initialization successful! This folder hive 800.dat
torch.Size([1, 1, 128, 128, 128]) torch.cuda.FloatTensor


In [12]:
unet3d.eval()
with torch.no_grad():
    pred = unet3d(ss)
pred = pred.squeeze().cpu().numpy().transpose()
print(pred.shape)
pred.tofile('data/demo/ori128.dat')

(128, 128, 128)


- 重建训练数据集

In [4]:
from sgykit import Sgykit
sgykit = Sgykit()
redb = sgykit.reconstruct_sgy('data/validation/fault/')

In [7]:
print(redb.shape)

(512, 640, 128)


In [6]:
from generatadata import save_data
save_data(redb, 'data/validation/fault/', 'redb.datt')